In [ ]:
from pyspark.sql import DataFrame
from pyspark.sql import SparkSession

In [ ]:
spark = SparkSession.builder.appName('my_first_app_name').getOrCreate()

In [ ]:
stringCSVRDD = spark.sparkContext.parallelize([
    (123, "490654 | 49065410 | 12476694 | 2019-06-26 | - | 2 | FPLOSRate | Success | Close:true, Ticket:50c82efe0944000e"),
    (234, "490654 | 49065410 | 12476694 | 2019-06-26 | 4N | 2 | FPLOSRate | Success | Close:true, Ticket:50c82efe0944000e"),
    (345, "490654 | 49065410 | 12476694 | 2019-06-26 | 4N | 2 | FPLOSRate | Success | Close:true, Ticket:50c82efe0944000e"),
    (345, "490656 | 49065410 | 12476694 | 2019-06-26 | 4N | 2 | FPLOSRate | Success | Close:true, Ticket:50c82efe0944000e"),
    (345, "490656 | 49065410 | 12476694 | 2019-06-26 | 4N | 2 | FPLOSRate | Success | Close:true, Ticket:50c82efe0944000e"),
    (345, "490656 | 49065410 | 12476694 | 2019-06-26 | 4N | 2 | FPLOSRate | Success | Close:true, Ticket:50c82efe0944000e"),
    (345, "490657 | 49065410 | 12476694 | 2019-06-26 | 4N | 2 | FPLOSRate | Success | Close:true, Ticket:50c82efe0944000e"),
    (345, "490658 | 49065410 | 12476694 | 2019-06-26 | 4N | 2 | FPLOSRate | Success | Close:true, Ticket:50c82efe0944000e")
])

In [142]:
from pyspark.sql.types import StructType, StructField, LongType, StringType

In [143]:
schema = StructType([
    StructField("id", LongType(), True),
    StructField("name", StringType(), True)
])

In [144]:
swimmers = spark.createDataFrame(stringCSVRDD,schema)

In [145]:
swimmers.registerTempTable("swimmers")

In [146]:
from pyspark.sql.functions import split,udf,trim

In [147]:
split_col = split(swimmers['name'], '\|')

swimmers = swimmers.withColumn('hotel', trim(split_col.getItem(0)))
swimmers = swimmers.withColumn('room_type', trim(split_col.getItem(1)))
swimmers = swimmers.withColumn('rate_plan', trim(split_col.getItem(2)))
swimmers = swimmers.withColumn('date', trim(split_col.getItem(3)))
swimmers = swimmers.withColumn('los', trim(split_col.getItem(4)))
swimmers = swimmers.withColumn('guest', trim(split_col.getItem(5)))
swimmers = swimmers.withColumn('event', trim(split_col.getItem(6)))
swimmers = swimmers.withColumn('result', trim(split_col.getItem(7)))
swimmers = swimmers.withColumn('event_detail', trim(split_col.getItem(8)))
swimmers = swimmers.withColumn('test', trim(split_col.getItem(9)))

In [148]:
swimmers.head(10)

[Row(id=123, name='490654 | 49065410 | 12476694 | 2019-06-26 |  | 2 | FPLOSRate | Success | Close:true, Ticket:50c82efe0944000e', hotel='490654', room_type='49065410', rate_plan='12476694', date='2019-06-26', los='', guest='2', event='FPLOSRate', result='Success', event_detail='Close:true, Ticket:50c82efe0944000e', test=None),
 Row(id=234, name='490654 | 49065410 | 12476694 | 2019-06-26 | 4N | 2 | FPLOSRate | Success | Close:true, Ticket:50c82efe0944000e', hotel='490654', room_type='49065410', rate_plan='12476694', date='2019-06-26', los='4N', guest='2', event='FPLOSRate', result='Success', event_detail='Close:true, Ticket:50c82efe0944000e', test=None),
 Row(id=345, name='490654 | 49065410 | 12476694 | 2019-06-26 | 4N | 2 | FPLOSRate | Success | Close:true, Ticket:50c82efe0944000e', hotel='490654', room_type='49065410', rate_plan='12476694', date='2019-06-26', los='4N', guest='2', event='FPLOSRate', result='Success', event_detail='Close:true, Ticket:50c82efe0944000e', test=None),
 Row(

In [ ]:
swimmers.groupby(swimmers.name)

In [165]:
row_list = swimmers.groupby(swimmers.hotel).count().collect()

In [161]:
def write_hotel_data(hotel_log_item):
    print(hotel_log_item.hotel)

In [ ]:
for item in row_list:
    #print(i['hotel'])
    print(swimmers.where(swimmers.hotel == item['hotel']).head(10))
    print("===============================================")

[Row(id=345, name='490657 | 49065410 | 12476694 | 2019-06-26 | 4N | 2 | FPLOSRate | Success | Close:true, Ticket:50c82efe0944000e', hotel='490657', room_type='49065410', rate_plan='12476694', date='2019-06-26', los='4N', guest='2', event='FPLOSRate', result='Success', event_detail='Close:true, Ticket:50c82efe0944000e', test=None)]
[Row(id=345, name='490658 | 49065410 | 12476694 | 2019-06-26 | 4N | 2 | FPLOSRate | Success | Close:true, Ticket:50c82efe0944000e', hotel='490658', room_type='49065410', rate_plan='12476694', date='2019-06-26', los='4N', guest='2', event='FPLOSRate', result='Success', event_detail='Close:true, Ticket:50c82efe0944000e', test=None)]
[Row(id=123, name='490654 | 49065410 | 12476694 | 2019-06-26 |  | 2 | FPLOSRate | Success | Close:true, Ticket:50c82efe0944000e', hotel='490654', room_type='49065410', rate_plan='12476694', date='2019-06-26', los='', guest='2', event='FPLOSRate', result='Success', event_detail='Close:true, Ticket:50c82efe0944000e', test=None), Row(i

In [162]:
swimmers.groupby(swimmers.hotel).count().foreach(write_hotel_data)

In [158]:
swimmers.where(swimmers.hotel == swimmers.groupby(swimmers.hotel).count().collect()[0]['hotel']).collect()

[Row(id=345, name='490657 | 49065410 | 12476694 | 2019-06-26 | 4N | 2 | FPLOSRate | Success | Close:true, Ticket:50c82efe0944000e', hotel='490657', room_type='49065410', rate_plan='12476694', date='2019-06-26', los='4N', guest='2', event='FPLOSRate', result='Success', event_detail='Close:true, Ticket:50c82efe0944000e', test=None)]

In [ ]:
swimmers.where(df.age == 2)

In [177]:
swimmers.take(1)[0]['hotel'] 

'490654'